<h1 align="center"> Predicting Titanic Survivors </h1>

In [3]:
#importing neccessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from numpy import mean
from numpy import std
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [ ]:
# 2. Logistic Regression on the Titanic Dataset

In [208]:
#reading the titanic dataset
titanic = pd.read_csv("/Users/sesh/Desktop/train.csv")

In [209]:
#dataset info
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  889 non-null    int64  
 1   Survived    889 non-null    int64  
 2   Pclass      889 non-null    int64  
 3   Age         889 non-null    float64
 4   SibSp       889 non-null    int64  
 5   Parch       889 non-null    int64  
 6   Fare        889 non-null    float64
 7   male        889 non-null    int64  
 8   Q           889 non-null    int64  
 9   S           889 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 69.6 KB


In [210]:
#formatting the target variable as categorical 
titanic["Survived"] = titanic["Survived"].astype("category")

In [211]:
#saving independent and target variables separately
X = titanic.drop('Survived', axis = 1)
Y = titanic['Survived'] 

In [212]:
#creating dummy variables for Pclass
pclass_dumm = pd.get_dummies(X['Pclass'],drop_first=True, prefix="Pclass")
pclass_dumm.head()

,Pclass_2,Pclass_3
0,0,1
1,0,0
2,0,1
3,0,0
4,0,1


In [213]:
#creating dummy variables for SibSp
SibSp_dumm = pd.get_dummies(X['SibSp'],drop_first=True,prefix="SibSp")
SibSp_dumm.head()

,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,0,0,0,0
3,1,0,0,0,0,0
4,0,0,0,0,0,0


In [214]:
#creating dummy variables for Parch
Parch_dumm = pd.get_dummies(X['Parch'],drop_first=True,prefix="Parch")
Parch_dumm.head()

,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [215]:
#dropping unnecessary columns for regression
X.drop(['Unnamed: 0','Pclass','SibSp','Parch'],axis=1,inplace=True)
X.head()

,Age,Fare,male,Q,S
0,22.0,7.2500,1,0,1
1,38.0,71.2833,0,0,0
2,26.0,7.9250,0,0,1
3,35.0,53.1000,0,0,1
4,35.0,8.0500,1,0,1


In [216]:
#concatenating dummy variables
X = pd.concat([X,pclass_dumm,SibSp_dumm,Parch_dumm],axis=1)
X.head()

,Age,Fare,male,Q,S,Pclass_2,Pclass_3,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,22.0,7.2500,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
1,38.0,71.2833,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,26.0,7.9250,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,35.0,53.1000,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,35.0,8.0500,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [217]:
# standardizing dependent variables
X1 = preprocessing.scale(X)

In [218]:
#Creating a base model for logistic regression with penalty of Ridge Regression
log = LogisticRegression(penalty='l2', dual= False)
log.fit(X1,Y)

LogisticRegression()

In [219]:
#creating a dictionary for hyperparameters
hyperparam = {
    "solver" : ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
    "max_iter" : [100, 1000, 10],
    "tol" : [0.1, 1e-2 ,1e-5, 1e-4, 1e-3],
    "C" : [0.1, 0.5, 1, 10]
}

In [220]:
###Model 1 with kfold value 2

In [221]:
#setting k cross value to 2
cv = KFold(n_splits=2)

In [222]:
#performing grid search on possible hyperparameters to obtain the best combination values
grid_log = GridSearchCV(estimator = log,
                         param_grid = hyperparam,
                         scoring = ["accuracy", "r2"],
                         cv=cv,
                         refit = "accuracy", 
                         verbose = 4
                         )

In [223]:
#fitting the grid model's to train data
grid_log.fit(X1, Y)

Fitting 2 folds for each of 300 candidates, totalling 600 fits
[CV 1/2] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=100, solver=lbfgs, tol=1e-05; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=100, solver=lbfgs, tol=1e-05; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=100, solver=lbfgs, tol=0.0001; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=100, solver=lbfgs, tol=0.0001; accuracy: (test=0.795) r2: (test=0.124) total time=   0.

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/sesh/opt/ana

[CV 2/2] END C=0.1, max_iter=100, solver=saga, tol=0.001; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.1; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.1; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.01; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.01; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=1000, solver=lbfgs, tol=1e-05; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=1000, solver=lbfgs, tol=1e-05; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.0001; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

[CV 2/2] END C=0.1, max_iter=10, solver=newton-cg, tol=0.01; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=10, solver=newton-cg, tol=1e-05; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=10, solver=newton-cg, tol=1e-05; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=10, solver=newton-cg, tol=0.0001; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=10, solver=newton-cg, tol=0.0001; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=10, solver=newton-cg, tol=0.001; accuracy: (test=0.780) r2: (test=0.075) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=10, solver=newton-cg, tol=0.001; accuracy: (test=0.795) r2: (test=0.124) total time=   0.0s
[CV 1/2] END C=0.1, max_iter=10, solver=liblinear, tol=0.1; accuracy: (test=0.784) r2: (test=0.094) total time=   0.0s
[CV 2/2] END C=0.1, max_iter=10, 

...


[CV 1/2] END C=10, max_iter=1000, solver=lbfgs, tol=0.001; accuracy: (test=0.782) r2: (test=0.085) total time=   0.0s
[CV 2/2] END C=10, max_iter=1000, solver=lbfgs, tol=0.001; accuracy: (test=0.791) r2: (test=0.105) total time=   0.0s
[CV 1/2] END C=10, max_iter=1000, solver=newton-cg, tol=0.1; accuracy: (test=0.782) r2: (test=0.085) total time=   0.0s
[CV 2/2] END C=10, max_iter=1000, solver=newton-cg, tol=0.1; accuracy: (test=0.791) r2: (test=0.105) total time=   0.0s
[CV 1/2] END C=10, max_iter=1000, solver=newton-cg, tol=0.01; accuracy: (test=0.782) r2: (test=0.085) total time=   0.0s
[CV 2/2] END C=10, max_iter=1000, solver=newton-cg, tol=0.01; accuracy: (test=0.791) r2: (test=0.105) total time=   0.0s
[CV 1/2] END C=10, max_iter=1000, solver=newton-cg, tol=1e-05; accuracy: (test=0.782) r2: (test=0.085) total time=   0.0s
[CV 2/2] END C=10, max_iter=1000, solver=newton-cg, tol=1e-05; accuracy: (test=0.793) r2: (test=0.115) total time=   0.0s
[CV 1/2] END C=10, max_iter=1000, sol

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


[CV 1/2] END C=10, max_iter=1000, solver=sag, tol=0.0001; accuracy: (test=0.782) r2: (test=0.085) total time=   0.0s
[CV 2/2] END C=10, max_iter=1000, solver=sag, tol=0.0001; accuracy: (test=0.791) r2: (test=0.105) total time=   0.0s
[CV 1/2] END C=10, max_iter=1000, solver=sag, tol=0.001; accuracy: (test=0.782) r2: (test=0.085) total time=   0.0s
[CV 2/2] END C=10, max_iter=1000, solver=sag, tol=0.001; accuracy: (test=0.791) r2: (test=0.105) total time=   0.0s
[CV 1/2] END C=10, max_iter=1000, solver=saga, tol=0.1; accuracy: (test=0.746) r2: (test=-0.066) total time=   0.0s
[CV 2/2] END C=10, max_iter=1000, solver=saga, tol=0.1; accuracy: (test=0.788) r2: (test=0.096) total time=   0.0s
[CV 1/2] END C=10, max_iter=1000, solver=saga, tol=0.01; accuracy: (test=0.784) r2: (test=0.094) total time=   0.0s
[CV 2/2] END C=10, max_iter=1000, solver=saga, tol=0.01; accuracy: (test=0.793) r2: (test=0.115) total time=   0.0s
[CV 1/2] END C=10, max_iter=1000, solver=saga, tol=1e-05; accuracy: (te

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             estimator=LogisticRegression(),
             param_grid={'C': [0.1, 0.5, 1, 10], 'max_iter': [100, 1000, 10],
                         'solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag',
                                    'saga'],
                         'tol': [0.1, 0.01, 1e-05, 0.0001, 0.001]},
             refit='accuracy', scoring=['accuracy', 'r2'], verbose=4)

In [224]:
#printing the parameters for best model
print(grid_log.best_estimator_)

LogisticRegression(C=10, solver='sag', tol=0.1)


In [225]:
#printing the parameters for best model
print(grid_log.best_params_)

{'C': 10, 'max_iter': 100, 'solver': 'sag', 'tol': 0.1}


In [226]:
#printing the score for best model
print(grid_log.best_score_)

0.7986461180281406


In [227]:
#creating logistic model with penalty of Ridge Regression and the values retrieved for the best parameters
log = LogisticRegression(penalty='l2', dual= False, C=10, max_iter=100, solver='sag', tol=0.1 )

#fitting the model over train set
log.fit(X1,Y)

LogisticRegression(C=10, solver='sag', tol=0.1)

In [228]:
#printing accuracy score for the model
score = cross_val_score(log, X1, Y, scoring= 'accuracy', cv = cv, n_jobs=-1)
print("Cross validatioin mean score: ", score.mean())

Cross validatioin mean score:  0.7896598846037048


In [ ]:
###Model 2 with kfold value 3

In [229]:
#setting k cross value to 3
cv = KFold(n_splits=3)

In [230]:
#performing grid search on possible hyperparameters to obtain the best combination values
grid_log = GridSearchCV(estimator = log,
                         param_grid = hyperparam,
                         scoring = ["accuracy", "r2"],
                         cv=cv,
                         refit = "accuracy", 
                         verbose = 4
                         )

In [231]:
#fitting the grid model's to train data
grid_log.fit(X1, Y)

Fitting 3 folds for each of 300 candidates, totalling 900 fits
[CV 1/3] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.805) r2: (test=0.146) total time=   0.0s
[CV 2/3] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.807) r2: (test=0.215) total time=   0.0s
[CV 3/3] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.811) r2: (test=0.180) total time=   0.0s
[CV 1/3] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.805) r2: (test=0.146) total time=   0.0s
[CV 2/3] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.807) r2: (test=0.215) total time=   0.0s
[CV 3/3] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.811) r2: (test=0.180) total time=   0.0s
[CV 1/3] END C=0.1, max_iter=100, solver=lbfgs, tol=1e-05; accuracy: (test=0.805) r2: (test=0.146) total time=   0.0s
[CV 2/3] END C=0.1, max_iter=100, solver=lbfgs, tol=1e-05; accuracy: (test=0.807) r2: (test=0.215) total time=   0.0s
[C

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


[CV 1/3] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.01; accuracy: (test=0.805) r2: (test=0.146) total time=   0.0s
[CV 2/3] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.01; accuracy: (test=0.807) r2: (test=0.215) total time=   0.0s
[CV 3/3] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.01; accuracy: (test=0.811) r2: (test=0.180) total time=   0.0s
[CV 1/3] END C=0.1, max_iter=1000, solver=lbfgs, tol=1e-05; accuracy: (test=0.805) r2: (test=0.146) total time=   0.0s
[CV 2/3] END C=0.1, max_iter=1000, solver=lbfgs, tol=1e-05; accuracy: (test=0.807) r2: (test=0.215) total time=   0.0s
[CV 3/3] END C=0.1, max_iter=1000, solver=lbfgs, tol=1e-05; accuracy: (test=0.811) r2: (test=0.180) total time=   0.0s
[CV 1/3] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.0001; accuracy: (test=0.805) r2: (test=0.146) total time=   0.0s
[CV 2/3] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.0001; accuracy: (test=0.807) r2: (test=0.215) total time=   0.0s
[CV 3/3] END C=0.1, max_iter=1000, solver=lbfgs, 

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

[CV 2/3] END C=0.1, max_iter=10, solver=lbfgs, tol=0.001; accuracy: (test=0.807) r2: (test=0.215) total time=   0.0s
[CV 3/3] END C=0.1, max_iter=10, solver=lbfgs, tol=0.001; accuracy: (test=0.811) r2: (test=0.180) total time=   0.0s
[CV 1/3] END C=0.1, max_iter=10, solver=newton-cg, tol=0.1; accuracy: (test=0.805) r2: (test=0.146) total time=   0.0s
[CV 2/3] END C=0.1, max_iter=10, solver=newton-cg, tol=0.1; accuracy: (test=0.807) r2: (test=0.215) total time=   0.0s
[CV 3/3] END C=0.1, max_iter=10, solver=newton-cg, tol=0.1; accuracy: (test=0.811) r2: (test=0.180) total time=   0.0s
[CV 1/3] END C=0.1, max_iter=10, solver=newton-cg, tol=0.01; accuracy: (test=0.805) r2: (test=0.146) total time=   0.0s
[CV 2/3] END C=0.1, max_iter=10, solver=newton-cg, tol=0.01; accuracy: (test=0.807) r2: (test=0.215) total time=   0.0s
[CV 3/3] END C=0.1, max_iter=10, solver=newton-cg, tol=0.01; accuracy: (test=0.811) r2: (test=0.180) total time=   0.0s
[CV 1/3] END C=0.1, max_iter=10, solver=newton-cg

...

[CV 1/3] END C=10, max_iter=1000, solver=saga, tol=1e-05; accuracy: (test=0.795) r2: (test=0.101) total time=   0.1s
[CV 2/3] END C=10, max_iter=1000, solver=saga, tol=1e-05; accuracy: (test=0.811) r2: (test=0.229) total time=   0.1s
[CV 3/3] END C=10, max_iter=1000, solver=saga, tol=1e-05; accuracy: (test=0.801) r2: (test=0.136) total time=   0.1s
[CV 1/3] END C=10, max_iter=1000, solver=saga, tol=0.0001; accuracy: (test=0.795) r2: (test=0.101) total time=   0.1s
[CV 2/3] END C=10, max_iter=1000, solver=saga, tol=0.0001; accuracy: (test=0.811) r2: (test=0.229) total time=   0.0s
[CV 3/3] END C=10, max_iter=1000, solver=saga, tol=0.0001; accuracy: (test=0.801) r2: (test=0.136) total time=   0.0s
[CV 1/3] END C=10, max_iter=1000, solver=saga, tol=0.001; accuracy: (test=0.795) r2: (test=0.101) total time=   0.0s
[CV 2/3] END C=10, max_iter=1000, solver=saga, tol=0.001; accuracy: (test=0.807) r2: (test=0.215) total time=   0.0s
[CV 3/3] END C=10, max_iter=1000, solver=saga, tol=0.001; acc

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

[CV 1/3] END C=10, max_iter=10, solver=newton-cg, tol=1e-05; accuracy: (test=0.795) r2: (test=0.101) total time=   0.0s
[CV 2/3] END C=10, max_iter=10, solver=newton-cg, tol=1e-05; accuracy: (test=0.811) r2: (test=0.229) total time=   0.0s
[CV 3/3] END C=10, max_iter=10, solver=newton-cg, tol=1e-05; accuracy: (test=0.801) r2: (test=0.136) total time=   0.0s
[CV 1/3] END C=10, max_iter=10, solver=newton-cg, tol=0.0001; accuracy: (test=0.795) r2: (test=0.101) total time=   0.0s
[CV 2/3] END C=10, max_iter=10, solver=newton-cg, tol=0.0001; accuracy: (test=0.811) r2: (test=0.229) total time=   0.0s
[CV 3/3] END C=10, max_iter=10, solver=newton-cg, tol=0.0001; accuracy: (test=0.801) r2: (test=0.136) total time=   0.0s
[CV 1/3] END C=10, max_iter=10, solver=newton-cg, tol=0.001; accuracy: (test=0.795) r2: (test=0.101) total time=   0.0s
[CV 2/3] END C=10, max_iter=10, solver=newton-cg, tol=0.001; accuracy: (test=0.811) r2: (test=0.229) total time=   0.0s
[CV 3/3] END C=10, max_iter=10, solve

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
             estimator=LogisticRegression(C=10, solver='sag', tol=0.1),
             param_grid={'C': [0.1, 0.5, 1, 10], 'max_iter': [100, 1000, 10],
                         'solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag',
                                    'saga'],
                         'tol': [0.1, 0.01, 1e-05, 0.0001, 0.001]},
             refit='accuracy', scoring=['accuracy', 'r2'], verbose=4)

In [232]:
#printing the parameters for best model
print(grid_log.best_estimator_)

LogisticRegression(C=0.1, solver='liblinear', tol=0.1)


In [233]:
#printing the parameters for best model
print(grid_log.best_params_)

{'C': 0.1, 'max_iter': 100, 'solver': 'liblinear', 'tol': 0.1}


In [234]:
#printing the score for best model
print(grid_log.best_score_)

0.8087822671156005


In [235]:
#creating logistic model with penalty of Ridge Regression and the values retrieved for the best parameters
log = LogisticRegression(penalty='l2', dual= False, C=0.1, max_iter=100, solver='liblinear', tol=0.1 )

#fitting the model over train set
log.fit(X1,Y)

LogisticRegression(C=0.1, solver='liblinear', tol=0.1)

In [236]:
#printing accuracy score for the model
score = cross_val_score(log, X1, Y, scoring= 'accuracy', cv = cv, n_jobs=-1)
print("Cross validatioin mean score: ", score.mean())

Cross validatioin mean score:  0.8087822671156005


In [237]:
###Model 3 with kfold value 4

In [238]:
#setting k cross value to 4
cv = KFold(n_splits=4)

In [239]:
#performing grid search on possible hyperparameters to obtain the best combination values
grid_log = GridSearchCV(estimator = log,
                         param_grid = hyperparam,
                         scoring = ["accuracy", "r2"],
                         cv=cv,
                         refit = "accuracy", 
                         verbose = 4
                         )

In [240]:
#fitting the grid model's to train data
grid_log.fit(X1, Y)

Fitting 4 folds for each of 300 candidates, totalling 1200 fits
[CV 1/4] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.821) r2: (test=0.207) total time=   0.0s
[CV 2/4] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.811) r2: (test=0.231) total time=   0.0s
[CV 3/4] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.797) r2: (test=0.142) total time=   0.0s
[CV 4/4] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.820) r2: (test=0.222) total time=   0.0s
[CV 1/4] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.821) r2: (test=0.207) total time=   0.0s
[CV 2/4] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.811) r2: (test=0.231) total time=   0.0s
[CV 3/4] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.797) r2: (test=0.142) total time=   0.0s
[CV 4/4] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.820) r2: (test=0.222) total time=   0.0s
[CV 

[CV 2/4] END C=0.1, max_iter=100, solver=saga, tol=0.1; accuracy: (test=0.797) r2: (test=0.176) total time=   0.0s
[CV 3/4] END C=0.1, max_iter=100, solver=saga, tol=0.1; accuracy: (test=0.784) r2: (test=0.085) total time=   0.0s
[CV 4/4] END C=0.1, max_iter=100, solver=saga, tol=0.1; accuracy: (test=0.811) r2: (test=0.184) total time=   0.0s
[CV 1/4] END C=0.1, max_iter=100, solver=saga, tol=0.01; accuracy: (test=0.816) r2: (test=0.187) total time=   0.0s
[CV 2/4] END C=0.1, max_iter=100, solver=saga, tol=0.01; accuracy: (test=0.811) r2: (test=0.231) total time=   0.0s
[CV 3/4] END C=0.1, max_iter=100, solver=saga, tol=0.01; accuracy: (test=0.793) r2: (test=0.123) total time=   0.0s
[CV 4/4] END C=0.1, max_iter=100, solver=saga, tol=0.01; accuracy: (test=0.820) r2: (test=0.222) total time=   0.0s
[CV 1/4] END C=0.1, max_iter=100, solver=saga, tol=1e-05; accuracy: (test=0.821) r2: (test=0.207) total time=   0.0s
[CV 2/4] END C=0.1, max_iter=100, solver=saga, tol=1e-05; accuracy: (test=

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


[CV 4/4] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.001; accuracy: (test=0.820) r2: (test=0.222) total time=   0.0s
[CV 1/4] END C=0.1, max_iter=1000, solver=newton-cg, tol=0.1; accuracy: (test=0.821) r2: (test=0.207) total time=   0.0s
[CV 2/4] END C=0.1, max_iter=1000, solver=newton-cg, tol=0.1; accuracy: (test=0.811) r2: (test=0.231) total time=   0.0s
[CV 3/4] END C=0.1, max_iter=1000, solver=newton-cg, tol=0.1; accuracy: (test=0.797) r2: (test=0.142) total time=   0.0s
[CV 4/4] END C=0.1, max_iter=1000, solver=newton-cg, tol=0.1; accuracy: (test=0.820) r2: (test=0.222) total time=   0.0s
[CV 1/4] END C=0.1, max_iter=1000, solver=newton-cg, tol=0.01; accuracy: (test=0.821) r2: (test=0.207) total time=   0.0s
[CV 2/4] END C=0.1, max_iter=1000, solver=newton-cg, tol=0.01; accuracy: (test=0.811) r2: (test=0.231) total time=   0.0s
[CV 3/4] END C=0.1, max_iter=1000, solver=newton-cg, tol=0.01; accuracy: (test=0.797) r2: (test=0.142) total time=   0.0s
[CV 4/4] END C=0.1, max_iter=10

[CV 3/4] END C=0.1, max_iter=1000, solver=saga, tol=0.001; accuracy: (test=0.797) r2: (test=0.142) total time=   0.0s
[CV 4/4] END C=0.1, max_iter=1000, solver=saga, tol=0.001; accuracy: (test=0.820) r2: (test=0.222) total time=   0.0s
[CV 1/4] END C=0.1, max_iter=10, solver=lbfgs, tol=0.1; accuracy: (test=0.821) r2: (test=0.207) total time=   0.0s
[CV 2/4] END C=0.1, max_iter=10, solver=lbfgs, tol=0.1; accuracy: (test=0.811) r2: (test=0.231) total time=   0.0s
[CV 3/4] END C=0.1, max_iter=10, solver=lbfgs, tol=0.1; accuracy: (test=0.797) r2: (test=0.142) total time=   0.0s
[CV 4/4] END C=0.1, max_iter=10, solver=lbfgs, tol=0.1; accuracy: (test=0.820) r2: (test=0.222) total time=   0.0s
[CV 1/4] END C=0.1, max_iter=10, solver=lbfgs, tol=0.01; accuracy: (test=0.821) r2: (test=0.207) total time=   0.0s
[CV 2/4] END C=0.1, max_iter=10, solver=lbfgs, tol=0.01; accuracy: (test=0.811) r2: (test=0.231) total time=   0.0s
[CV 3/4] END C=0.1, max_iter=10, solver=lbfgs, tol=0.01; accuracy: (test

...

[CV 1/4] END C=10, max_iter=1000, solver=saga, tol=0.0001; accuracy: (test=0.816) r2: (test=0.187) total time=   0.0s
[CV 2/4] END C=10, max_iter=1000, solver=saga, tol=0.0001; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/4] END C=10, max_iter=1000, solver=saga, tol=0.0001; accuracy: (test=0.802) r2: (test=0.161) total time=   0.0s
[CV 4/4] END C=10, max_iter=1000, solver=saga, tol=0.0001; accuracy: (test=0.833) r2: (test=0.281) total time=   0.0s
[CV 1/4] END C=10, max_iter=1000, solver=saga, tol=0.001; accuracy: (test=0.812) r2: (test=0.167) total time=   0.0s
[CV 2/4] END C=10, max_iter=1000, solver=saga, tol=0.001; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/4] END C=10, max_iter=1000, solver=saga, tol=0.001; accuracy: (test=0.797) r2: (test=0.142) total time=   0.0s
[CV 4/4] END C=10, max_iter=1000, solver=saga, tol=0.001; accuracy: (test=0.833) r2: (test=0.281) total time=   0.0s
[CV 1/4] END C=10, max_iter=10, solver=lbfgs, tol=0.1; accur

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

[CV 4/4] END C=10, max_iter=10, solver=lbfgs, tol=1e-05; accuracy: (test=0.833) r2: (test=0.281) total time=   0.0s
[CV 1/4] END C=10, max_iter=10, solver=lbfgs, tol=0.0001; accuracy: (test=0.816) r2: (test=0.187) total time=   0.0s
[CV 2/4] END C=10, max_iter=10, solver=lbfgs, tol=0.0001; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/4] END C=10, max_iter=10, solver=lbfgs, tol=0.0001; accuracy: (test=0.802) r2: (test=0.161) total time=   0.0s
[CV 4/4] END C=10, max_iter=10, solver=lbfgs, tol=0.0001; accuracy: (test=0.833) r2: (test=0.281) total time=   0.0s
[CV 1/4] END C=10, max_iter=10, solver=lbfgs, tol=0.001; accuracy: (test=0.816) r2: (test=0.187) total time=   0.0s
[CV 2/4] END C=10, max_iter=10, solver=lbfgs, tol=0.001; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/4] END C=10, max_iter=10, solver=lbfgs, tol=0.001; accuracy: (test=0.802) r2: (test=0.161) total time=   0.0s
[CV 4/4] END C=10, max_iter=10, solver=lbfgs, tol=0.001; accuracy: (

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_

GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=False),
             estimator=LogisticRegression(C=0.1, solver='liblinear', tol=0.1),
             param_grid={'C': [0.1, 0.5, 1, 10], 'max_iter': [100, 1000, 10],
                         'solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag',
                                    'saga'],
                         'tol': [0.1, 0.01, 1e-05, 0.0001, 0.001]},
             refit='accuracy', scoring=['accuracy', 'r2'], verbose=4)

In [241]:
#printing the parameters for best model
print(grid_log.best_estimator_)

LogisticRegression(C=10, solver='newton-cg', tol=1e-05)


In [242]:
#printing the parameters for best model
print(grid_log.best_params_)

{'C': 10, 'max_iter': 100, 'solver': 'newton-cg', 'tol': 1e-05}


In [243]:
#printing the score for best model
print(grid_log.best_score_)

0.8177746131782007


In [244]:
#creating logistic model with penalty of Ridge Regression and the values retrieved for the best parameters
log = LogisticRegression(penalty='l2', dual= False, C=10, max_iter=100, solver='newton-cg', tol= 1e-5 )

#fitting the model over train set
log.fit(X1,Y)

LogisticRegression(C=10, solver='newton-cg', tol=1e-05)

In [245]:
#printing accuracy score for the model
score = cross_val_score(log, X1, Y, scoring= 'accuracy', cv = cv, n_jobs=-1)
print("Cross validatioin mean score: ", score.mean())

Cross validatioin mean score:  0.8177746131782007


In [246]:
###Model 4 with kfold value 5

In [247]:
#creating a dictionary for hyperparameters
hyperparam = {
    "solver" : ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
    "max_iter" : [100, 1000, 10],
    "tol" : [0.1, 1e-2 ,1e-5, 1e-4, 1e-3],
    "C" : [0.1, 0.5, 1, 10]
}

In [248]:
#setting k cross value to 5
cv = KFold(n_splits=5)

#performing grid search on possible hyperparameters to obtain the best combination values
grid_log = GridSearchCV(estimator = log,
                         param_grid = hyperparam,
                         scoring = ["accuracy", "r2"],
                         cv=cv,
                         refit = "accuracy", 
                         verbose = 4
                         )

In [249]:
#fitting the grid model's to train data
grid_log.fit(X1, Y)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
[CV 1/5] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.809) r2: (test=0.130) total time=   0.0s
[CV 2/5] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/5] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.798) r2: (test=0.148) total time=   0.0s
[CV 4/5] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.787) r2: (test=0.114) total time=   0.0s
[CV 5/5] END C=0.1, max_iter=100, solver=lbfgs, tol=0.1; accuracy: (test=0.836) r2: (test=0.280) total time=   0.0s
[CV 1/5] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.809) r2: (test=0.130) total time=   0.0s
[CV 2/5] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/5] END C=0.1, max_iter=100, solver=lbfgs, tol=0.01; accuracy: (test=0.798) r2: (test=0.148) total time=   0.0s
[CV 4

[CV 4/5] END C=0.1, max_iter=100, solver=sag, tol=1e-05; accuracy: (test=0.792) r2: (test=0.137) total time=   0.0s
[CV 5/5] END C=0.1, max_iter=100, solver=sag, tol=1e-05; accuracy: (test=0.836) r2: (test=0.280) total time=   0.0s
[CV 1/5] END C=0.1, max_iter=100, solver=sag, tol=0.0001; accuracy: (test=0.809) r2: (test=0.130) total time=   0.0s
[CV 2/5] END C=0.1, max_iter=100, solver=sag, tol=0.0001; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/5] END C=0.1, max_iter=100, solver=sag, tol=0.0001; accuracy: (test=0.798) r2: (test=0.148) total time=   0.0s
[CV 4/5] END C=0.1, max_iter=100, solver=sag, tol=0.0001; accuracy: (test=0.792) r2: (test=0.137) total time=   0.0s
[CV 5/5] END C=0.1, max_iter=100, solver=sag, tol=0.0001; accuracy: (test=0.836) r2: (test=0.280) total time=   0.0s
[CV 1/5] END C=0.1, max_iter=100, solver=sag, tol=0.001; accuracy: (test=0.809) r2: (test=0.130) total time=   0.0s
[CV 2/5] END C=0.1, max_iter=100, solver=sag, tol=0.001; accuracy: 

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


[CV 2/5] END C=0.1, max_iter=100, solver=saga, tol=0.001; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/5] END C=0.1, max_iter=100, solver=saga, tol=0.001; accuracy: (test=0.798) r2: (test=0.148) total time=   0.0s
[CV 4/5] END C=0.1, max_iter=100, solver=saga, tol=0.001; accuracy: (test=0.787) r2: (test=0.114) total time=   0.0s
[CV 5/5] END C=0.1, max_iter=100, solver=saga, tol=0.001; accuracy: (test=0.836) r2: (test=0.280) total time=   0.0s
[CV 1/5] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.1; accuracy: (test=0.809) r2: (test=0.130) total time=   0.0s
[CV 2/5] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.1; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/5] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.1; accuracy: (test=0.798) r2: (test=0.148) total time=   0.0s
[CV 4/5] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.1; accuracy: (test=0.787) r2: (test=0.114) total time=   0.0s
[CV 5/5] END C=0.1, max_iter=1000, solver=lbfgs, tol=0.1; accura

[CV 5/5] END C=0.1, max_iter=1000, solver=sag, tol=1e-05; accuracy: (test=0.836) r2: (test=0.280) total time=   0.0s
[CV 1/5] END C=0.1, max_iter=1000, solver=sag, tol=0.0001; accuracy: (test=0.809) r2: (test=0.130) total time=   0.0s
[CV 2/5] END C=0.1, max_iter=1000, solver=sag, tol=0.0001; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/5] END C=0.1, max_iter=1000, solver=sag, tol=0.0001; accuracy: (test=0.798) r2: (test=0.148) total time=   0.0s
[CV 4/5] END C=0.1, max_iter=1000, solver=sag, tol=0.0001; accuracy: (test=0.792) r2: (test=0.137) total time=   0.0s
[CV 5/5] END C=0.1, max_iter=1000, solver=sag, tol=0.0001; accuracy: (test=0.836) r2: (test=0.280) total time=   0.0s
[CV 1/5] END C=0.1, max_iter=1000, solver=sag, tol=0.001; accuracy: (test=0.809) r2: (test=0.130) total time=   0.0s
[CV 2/5] END C=0.1, max_iter=1000, solver=sag, tol=0.001; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/5] END C=0.1, max_iter=1000, solver=sag, tol=0.001; a

...

[CV 5/5] END C=10, max_iter=10, solver=newton-cg, tol=0.01; accuracy: (test=0.853) r2: (test=0.355) total time=   0.0s
[CV 1/5] END C=10, max_iter=10, solver=newton-cg, tol=1e-05; accuracy: (test=0.815) r2: (test=0.156) total time=   0.0s
[CV 2/5] END C=10, max_iter=10, solver=newton-cg, tol=1e-05; accuracy: (test=0.820) r2: (test=0.272) total time=   0.0s
[CV 3/5] END C=10, max_iter=10, solver=newton-cg, tol=1e-05; accuracy: (test=0.792) r2: (test=0.124) total time=   0.0s
[CV 4/5] END C=10, max_iter=10, solver=newton-cg, tol=1e-05; accuracy: (test=0.787) r2: (test=0.114) total time=   0.0s
[CV 5/5] END C=10, max_iter=10, solver=newton-cg, tol=1e-05; accuracy: (test=0.853) r2: (test=0.355) total time=   0.0s
[CV 1/5] END C=10, max_iter=10, solver=newton-cg, tol=0.0001; accuracy: (test=0.815) r2: (test=0.156) total time=   0.0s
[CV 2/5] END C=10, max_iter=10, solver=newton-cg, tol=0.0001; accuracy: (test=0.820) r2: (test=0.272) total time=   0.0s
[CV 3/5] END C=10, max_iter=10, solver=

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/opti

[CV 5/5] END C=10, max_iter=10, solver=sag, tol=0.01; accuracy: (test=0.825) r2: (test=0.230) total time=   0.0s
[CV 1/5] END C=10, max_iter=10, solver=sag, tol=1e-05; accuracy: (test=0.809) r2: (test=0.130) total time=   0.0s
[CV 2/5] END C=10, max_iter=10, solver=sag, tol=1e-05; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/5] END C=10, max_iter=10, solver=sag, tol=1e-05; accuracy: (test=0.792) r2: (test=0.124) total time=   0.0s
[CV 4/5] END C=10, max_iter=10, solver=sag, tol=1e-05; accuracy: (test=0.787) r2: (test=0.114) total time=   0.0s
[CV 5/5] END C=10, max_iter=10, solver=sag, tol=1e-05; accuracy: (test=0.825) r2: (test=0.230) total time=   0.0s
[CV 1/5] END C=10, max_iter=10, solver=sag, tol=0.0001; accuracy: (test=0.809) r2: (test=0.130) total time=   0.0s
[CV 2/5] END C=10, max_iter=10, solver=sag, tol=0.0001; accuracy: (test=0.815) r2: (test=0.249) total time=   0.0s
[CV 3/5] END C=10, max_iter=10, solver=sag, tol=0.0001; accuracy: (test=0.792) r2: (tes

/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/sesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(C=10, solver='newton-cg', tol=1e-05),
             param_grid={'C': [0.1, 0.5, 1, 10], 'max_iter': [100, 1000, 10],
                         'solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag',
                                    'saga'],
                         'tol': [0.1, 0.01, 1e-05, 0.0001, 0.001]},
             refit='accuracy', scoring=['accuracy', 'r2'], verbose=4)

In [250]:
#printing the parameters for best model
print(grid_log.best_estimator_)

LogisticRegression(C=1, tol=0.01)


In [251]:
#printing the parameters for best model
print(grid_log.best_params_)

{'C': 1, 'max_iter': 100, 'solver': 'lbfgs', 'tol': 0.01}


In [252]:
#printing the score for best model
print(grid_log.best_score_)

0.8144416936456548


In [253]:
#creating logistic model with penalty of Ridge Regression and the values retrieved for the best parameters
log = LogisticRegression(penalty='l2', dual= False, C=1, max_iter=100, solver='lbfgs', tol=0.01 )

#fitting the model over train set
log.fit(X1,Y)

LogisticRegression(C=1, tol=0.01)

In [254]:
#printing accuracy score for the model
score = cross_val_score(log, X1, Y, scoring= 'accuracy', cv = cv, n_jobs=-1)
print("Cross validatioin mean score: ", score.mean())

Cross validatioin mean score:  0.8144416936456548


In [ ]:
#######Model 4 where k =5, has the highest accuracy with parameters set as {'C': 1, 'max_iter': 100, 'solver': 'lbfgs', 'tol': 0.01}